In [1]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-09


In [2]:
%run ./common_utils.ipynb

# Initial Functions

In [3]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [4]:
def load_df(file_name):
    df = pd.DataFrame()
    for i in [2021, 2022, 2023, 2024, 2025]:
        df_temp = pd.read_csv(f"../tables/{i}/{file_name}.csv")
        df_temp['Season'] = i
        df = pd.concat([df, df_temp])
        
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df.Date)
    if file_name == "season_gamelogs":
        df = df[~df[['Date', 'Team', 'Player']].duplicated(keep='last')]
    
    return df

In [5]:
def create_base_df():
    
    # Load dfs
    df = load_df('parlay_stats')
    df2 = load_df('nba_schedule')
    df3 = load_df('season_gamelogs')
    df4 = load_df('injuries')
    df5 = load_df('plyr_pos_xref')
    df6 = load_df('daily_lineups')
    gmlog_cols = ['game_id', 'Player', 'MP', 'PF']
    df7 = load_df('h1_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_h1", "PF": "PF_h1"})
    df8 = load_df('h2_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_h2", "PF": "PF_h2"})
    df9 = load_df('q1_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q1", "PF": "PF_q1"})
    df10 = load_df('q2_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q2", "PF": "PF_q2"})
    df11 = load_df('q3_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q3", "PF": "PF_q3"})
    df12 = load_df('q4_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q4", "PF": "PF_q4"})

    df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3['PR'] = df3.PTS + df3.REB 
    df3['PA'] = df3.PTS + df3.AST
    df3['RA'] = df3.REB + df3.AST
    df3['PRA'] = df3.PTS + df3.REB + df3.AST
    df3['STL_BLK'] = df3.STL + df3.BLK
    df = df.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.AwayPTS - df_mtch.HomePTS, df_mtch.HomePTS - df_mtch.AwayPTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)

    df6['role'] = 1
    df = df.merge(df6.drop('Pos', axis=1), on=['Season', 'Date', 'Team', 'Player'], how='left')
    df['role'] = df.role.fillna(2).astype(int)
    df['role'] = np.where(((df.MP < 8) & (df.role == 2)), 3, df.role)

    # Add gmlog splits
    df_gmlog_comb = df7.merge(df8, on=['game_id', 'Player'])
    for df_loop in (df9, df10, df11, df12):
        df_gmlog_comb = df_gmlog_comb.merge(df_loop, on=['game_id', 'Player'])
    df = df.merge(df_gmlog_comb, on=['game_id', 'Player'], how='left')
    
    global team_encoder, player_encoder, team_type_encoder, position_encoder, status_encoder
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()

    # Encode string cols
    team_encoder.fit(pd.concat([df["Team"], df["Opp"]], axis=0))
    players_fits = pd.concat([df["Player"], df3["Player"]], axis=0)
    players_fits = pd.concat([players_fits, df4["Player"]], axis=0).drop_duplicates()
    player_encoder.fit(players_fits)
    df["Team"] = team_encoder.transform(df["Team"])
    df["Opp"] = team_encoder.transform(df["Opp"])
    df["Player_name"] = df.Player
    df["Player"] = player_encoder.transform(df["Player"])
    df["Pos"] = position_encoder.fit_transform(df["Pos"])
    df['Team_type'] = team_type_encoder.fit_transform(df['Team_type'])
    df["Status"] = status_encoder.fit_transform(df["Status"])

    return df

### Create missing_df

In [6]:
def create_df_missing(df, pred_col):
    
    df3 = load_df('season_gamelogs')
    df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3['PR'] = df3.PTS + df3.REB 
    df3['PA'] = df3.PTS + df3.AST
    df3['RA'] = df3.REB + df3.AST
    df3['PRA'] = df3.PTS + df3.REB + df3.AST
    df3['STL_BLK'] = df3.STL + df3.BLK
    df4 = load_df('injuries')
    # Fill missing games from injuries.csv
    df_pred = create_base_df()
    team_games = df_pred[['Season', 'Team', 'Date']].drop_duplicates()
    players = df_pred[['Season','Player','Team']].drop_duplicates()
    fabricated = (players.sort_values('Season').groupby('Player', as_index=False).last())
    fabricated['Season'] = fabricated['Season'] + 1
    players = pd.concat([players, fabricated], ignore_index=True).drop_duplicates(['Season','Player','Team'])
    expanded = team_games.merge(players, on=['Season', 'Team'], how='left')

    df3["Team"] = team_encoder.transform(df3["Team"])
    df3["Player"] = player_encoder.transform(df3["Player"])
    df4["Team"] = team_encoder.transform(df4["Team"])
    df4["Player"] = player_encoder.transform(df4["Player"])
    df5 = load_df('plyr_pos_xref')
    df5['Team'] = team_encoder.transform(df5["Team"])
    df5['Player'] = player_encoder.transform(df5["Player"])

    expanded = expanded.merge(df3[['Season', 'Player', 'Date', 'MP']], on=['Season', 'Player', 'Date'], how='left').drop_duplicates(['Season', 'Date', 'Player', 'Team'])
    expanded = expanded[(expanded.MP.isnull()) & (expanded.Date != now)].drop('MP', axis=1)
    expanded = pd.concat([expanded, df4[df4.Status == 'Out'][['Season', 'Team', 'Date', 'Player']]])
    df4 = df4.merge(expanded, on=['Season', 'Date', 'Team', 'Player'], how='right')

    # Grab outs from players season gamelogs
    df4 = df4.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='outer')
    df4['Status'] = np.where(((df4.Active == 1) | (df4.MP > 0)), 'Available', df4.Status)
    df4['Status'] = np.where(((df4.Active == 0) | (df4.MP == 0) | (df4.MP.isnull())), 'Out', df4.Status)
    df4['Status'] = np.where((df4.Status == 'Out') & (df4.MP > 0), 'Available', df4.Status)
    df4['Status'] = np.where((df4.Status != 'Out') & (df4.MP == 0), 'Out', df4.Status)
    df4 = df4[df4.Status == 'Out'][['Season', 'Date', 'Team', 'Player']].drop_duplicates()
    
    df_missing = df[['Season', 'Date', 'Team', 'Player', 'role', pred_col]].copy()
    df_missing[f'{pred_col}_L10'] = (
        df_missing.groupby(['Player','Season'])[pred_col]
                  .transform(lambda x: x.rolling(10, min_periods=1).mean())
    )
    df_missing['role_L10_mode'] = (
        df_missing
            .groupby(['Player', 'Season'])['role']
            .transform(lambda x: x.rolling(10, min_periods=1)
                            .apply(lambda y: np.bincount(y.astype(np.int8), minlength=4).argmax(), raw=True))
    )
    df_missing = pd.merge_asof(df4, df_missing[["Season", "Player", "Date", "role", "role_L10_mode", f"{pred_col}_L10"]], 
                      on="Date", by=["Player", "Season"], direction="backward", allow_exact_matches=True).dropna()   
    df_missing = df_missing.merge(df5, on=['Season', 'Team', 'Player'])
    df_missing["Player"] = player_encoder.inverse_transform(df_missing["Player"])
    
    out_minutes = (
    df_missing
      .groupby(["Season", "Date", "Team"])
      .agg(
          starters_out=("role_L10_mode", lambda x: (x == 1).sum())
      )
      .reset_index()
    )

    return out_minutes

# Minutes Projection Model

In [11]:
def setup_df_mins(con, df):

    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'role', 'B2B', 
             'MP', 'MP_h1', 'MP_h2', 'MP_q1', 'MP_q2', 'MP_q3', 'MP_q4', 
             'PF', 'PF_h1', 'PF_h2', 'PF_q1', 'PF_q2', 'PF_q3', 'PF_q4', 
             'Spread', 'team_game_num', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP']:
        for N in [3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f'prev_team_mins_pct_L{N}'] = df[f'{col}_L{N}_avg'] / 240

    games_last_7_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('7D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L7_days"})
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['gms_L7_days'] = df.gms_L7_days.fillna(0).astype(int)
        
    df['role'] = np.where(((df.MP >= 24) & (df.role != 1)), 1, df.role)
    df['role'] = np.where(((df.MP < 24) & (df.role == 1)), 2, df.role)
    df['role'] = np.where(((df.MP < 14) & (df.role == 2)), 3, df.role)
    for N in [1, 3, 5]:
        df[f"recent_role_L{N}"] = (
            df
            .groupby(["Player", "Season"])["role"]
            .rolling(5, min_periods=1)
            .apply(lambda arr: np.bincount(arr.astype(int), minlength=4).argmax(), raw=True)
            .reset_index(level=[0, 1], drop=True)
        )
    df['missed_games'] = (df.groupby(['Player', 'Team', 'Season'])['team_game_num'].diff().sub(1).fillna(0).astype(int))
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread) < 7, 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread) >= 7) & (abs(df.Spread) <= 12), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread) > 12, 3, df.game_spread_type) 
    df['game_spread_type'] = np.where(df.is_OT > 0, 1, df.game_spread_type) 

    
#     df['game_spread_type'] = np.where(abs(df.Spread) < 4, 1, df.game_spread_type) 
#     df['game_spread_type'] = np.where((abs(df.Spread) >= 4) & (abs(df.Spread) <= 10), 2, df.game_spread_type) 
#     df['game_spread_type'] = np.where((abs(df.Spread) >= 11) & (abs(df.Spread) <= 15), 3, df.game_spread_type) 
#     df['game_spread_type'] = np.where(abs(df.Spread) > 16, 4, df.game_spread_type) 
#     df['game_spread_type'] = np.where(df.is_OT > 0, 1, df.game_spread_type) 

    # Tell model games exist after players injuries/susp
    team_games = df[['Season', 'Team', 'Date', 'team_game_num']].drop_duplicates()
    players = df[['Season','Player','Team']].drop_duplicates()
    fabricated = (players.sort_values('Season').groupby('Player', as_index=False).last())
    fabricated['Season'] = fabricated['Season'] + 1
    players = pd.concat([players, fabricated], ignore_index=True).drop_duplicates(['Season','Player','Team'])
    expanded = team_games.merge(players, on=['Season', 'Team'], how='left')
    expanded = expanded.merge(df[['Season', 'Player', 'Date', 'MP']], on=['Season', 'Player', 'Date'], how='left').drop_duplicates(['Season', 'Date', 'Player', 'Team'])
    expanded['player_played'] = expanded['MP'].notna().astype(int)
    expanded['team_played_no_player'] = ((expanded['player_played'] == 0)).astype(int)
    expanded['tm_plays_after'] = (expanded.groupby(['Player'])['team_played_no_player'].shift(-1))
    expanded['missed_gms_aftr'] = 0
    expanded['missed_gms_aftr'] = np.where((expanded.player_played == 1) & (expanded.tm_plays_after == 1), 1, expanded.missed_gms_aftr)
    df = df.merge(expanded[['Date', 'Team', 'Player', 'missed_gms_aftr']], on=['Date', 'Team', 'Player'])
    
    df2 = create_df_missing(df, 'MP')
    df = df.merge(df2, on=["Season", "Date", "Team"], how='left')
#     df['starters_out'] = df.starters_out.fillna(0)

    df['MP_change_pct_L10'] = (df['MP'] - df['MP_L10_avg']) / df['MP_L10_avg']
    MP_Inc_conds = (
                    ((df.role == 1) & (df['MP_change_pct_L10'] >= 0.05)) | 
                    ((df.role == 2) & (df['MP_change_pct_L10'] >= 0.10)) | 
                    ((df.role == 3) & (df['MP_change_pct_L10'] >= 0.15))
                   )
    df['MP_Increase'] = np.where(MP_Inc_conds, 1, 0)
    Early_Stop_conds = (
        ((df.role == 1) & (df['MP_change_pct_L10'] <= -0.25)) | 
        ((df.role == 2) & (df['MP_change_pct_L10'] <= -0.35)) | 
        ((df.role == 3) & (df['MP_change_pct_L10'] <= -0.45)) | 
        ((df.role != 3) & (df['MP'] < 8)) | 
        ((df.role == 1) & (df.MP_q4 == 0))
    )
    df['Early_Stop'] = np.where(Early_Stop_conds, 1, 0)
    Early_exit_conds = (
        ((df.MP_q2 + df.MP_h2) == 0 & (df.role == 1))
    )
    df['Early_Exit'] = np.where(Early_exit_conds, 1, 0)
    
#     Foul_Trouble_cond = (
#         (df.PF_q1 >= 2) |
#         (df.PF_h1 >= 3) |
#         ((df.PF_h1 + df.PF_q3) >= 4) |
#         (df.PF_q4 >= 5)
#     )
#     df['Foul_Trouble'] = np.where(Foul_Trouble_cond, 1, 0)
    
    df['Injured'] = ((df.Early_Stop == 1) & (df.missed_gms_aftr > 0)).astype(int)
    df['return_game'] = ((df.groupby('Player')['Injured'].shift(1) == 1) & (df.missed_games > 0)).astype(int)
    df['games_since_return'] = (df.groupby('Player')['return_game'].cumsum())
    df['games_since_return'] = (df.groupby(['Player', 'games_since_return']).cumcount())
    df['ramp_phase'] = 0
    df.loc[df.games_since_return == 0, 'ramp_phase'] = 1
    df.loc[df.games_since_return.isin([1, 2, 3]), 'ramp_phase'] = 2

    df = df.drop(['Season', 'Team_type', 'team_game_num', 'Spread', 'is_OT', 
                  'MP_h1', 'MP_h2', 'MP_q1', 'MP_q2', 'MP_q3', 'MP_q4',
                  'PF', 'PF_h1', 'PF_h2', 'PF_q1', 'PF_q2', 'PF_q3', 'PF_q4', 
                  'missed_gms_aftr', 'games_since_return', 'return_game', 'MP_change_pct_L10'], axis=1)      
    return df

# Main Model

In [12]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = ['TPM', 'FG', 'FT', 'TPA', 'FGA', 'FTA']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TPM', 'FG']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [13]:
def generate_predictions(tgt_stat):
    
    df_pred = create_base_df()
        
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    # Setup results
    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])
    df_pred = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]
    df_pred['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    df_pred['Diff2'] = abs((df_pred['MP'] - df_pred['MP_L5_avg']))
    df_pred = df_pred.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    
#     if df_pred.shape[0] >= 50:
#         print(df_pred.shape[0], 'rows')
#         for tm in df_pred.Team.unique():
#             display(df_pred[df_pred.Team == tm])
#     else:
#         display(df_pred)

    tds_picks = df_pred[['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_L5_avg', 'Spread', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [14]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

130 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
48,OKC,Aaron Wiggins,SG,MEM,24.249126,24.020999,-5.5,16.5,6.783350
36,OKC,Jalen Williams,SG,MEM,29.743721,29.722245,-5.5,24.5,14.923714
49,OKC,Isaiah Joe,SG,MEM,15.333146,15.737690,-5.5,12.5,4.343889
44,OKC,Ajay Mitchell,SG,MEM,25.732578,26.531894,-5.5,18.5,10.496835
99,OKC,Kenrich Williams,PF,MEM,14.779765,11.635395,-5.5,8.5,4.815668
54,OKC,Alex Caruso,SG,MEM,16.540630,17.293297,-5.5,6.5,4.659961
75,OKC,Luguentz Dort,SF,MEM,25.601997,24.958844,-5.5,9.5,7.885197


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
62,GSW,Jimmy Butler,SF,SAC,22.520132,29.925527,-14.5,20.5,11.892233
19,GSW,De'Anthony Melton,PG,SAC,19.030151,24.937706,-14.5,11.5,6.986497
34,GSW,Brandin Podziemski,SG,SAC,21.004318,23.912698,-14.5,9.5,5.500242
52,GSW,Gary Payton II,SG,SAC,14.958747,13.337201,-14.5,5.5,8.435448
39,GSW,Moses Moody,SG,SAC,24.293818,24.822355,-14.5,9.5,7.069699
6,GSW,Stephen Curry,PG,SAC,31.115688,30.641709,-14.5,28.5,26.110073
92,GSW,Quinten Post,PF,SAC,23.346830,17.054179,-14.5,8.5,10.387144
91,GSW,Draymond Green,PF,SAC,24.210556,23.673891,-14.5,8.5,7.069017
120,GSW,Al Horford,C,SAC,16.309765,16.041448,-14.5,6.5,7.505270


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
3,DEN,Jamal Murray,PG,ATL,23.665972,38.058349,1.5,26.5,17.946960
72,DEN,Peyton Watson,SF,ATL,33.887478,35.478035,1.5,17.5,23.795008
83,DEN,Aaron Gordon,PF,ATL,20.810839,20.411715,1.5,16.5,10.348672
46,DEN,Christian Braun,SG,ATL,26.726366,24.887005,1.5,12.5,7.247925
37,DEN,Tim Hardaway Jr.,SG,ATL,22.136652,30.475932,1.5,14.5,11.283556


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
10,NOP,Jordan Poole,PG,WAS,21.346910,26.947023,-2.5,17.5,10.007999
81,NOP,Zion Williamson,PF,WAS,26.999155,30.571741,-2.5,26.5,19.722078
73,NOP,Trey Murphy III,SF,WAS,34.782333,31.494416,-2.5,20.5,24.921597
53,NOP,Micah Peavy,SG,WAS,15.487043,18.939748,-2.5,6.5,4.028096
12,NOP,Jeremiah Fears,PG,WAS,25.235624,26.327991,-2.5,14.5,12.451155
107,NOP,Derik Queen,C,WAS,26.084295,27.126340,-2.5,13.5,11.990783
125,NOP,Yves Missi,C,WAS,16.550713,20.051783,-2.5,5.5,5.615215


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
94,LAL,Jake LaRavia,PF,MIL,31.546650,32.897797,-3.5,11.5,14.277228
66,LAL,LeBron James,SF,MIL,31.586842,32.184712,-3.5,22.5,25.136944
129,LAL,Jaxson Hayes,C,MIL,16.926165,17.493556,-3.5,4.5,5.502923
101,LAL,Jarred Vanderbilt,PF,MIL,20.362541,23.776580,-3.5,5.5,4.932251
47,LAL,Marcus Smart,SG,MIL,29.058493,32.480305,-3.5,8.5,8.805513
5,LAL,Luka Doncic,PG,MIL,34.590805,38.044213,-3.5,34.5,34.657593
112,LAL,Deandre Ayton,C,MIL,28.591944,30.537749,-3.5,12.5,12.460679


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
63,BOS,Jaylen Brown,SF,TOR,29.731071,35.701881,-10.5,28.5,25.748009
28,BOS,Derrick White,SG,TOR,34.424366,36.364411,-10.5,18.5,16.573223
40,BOS,Anfernee Simons,SG,TOR,20.487001,27.111554,-10.5,13.5,11.779771
95,BOS,Sam Hauser,PF,TOR,24.384872,22.364578,-10.5,8.5,6.836058
15,BOS,Payton Pritchard,PG,TOR,33.207878,33.549710,-10.5,16.5,15.276359
126,BOS,Luka Garza,C,TOR,17.974625,21.049144,-10.5,8.5,9.376067
119,BOS,Neemias Queta,C,TOR,25.091715,24.767548,-10.5,9.5,9.387689


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
23,PHO,Devin Booker,SG,NYK,31.706173,30.952489,1.5,26.5,23.914925
115,PHO,Mark Williams,C,NYK,24.498665,21.457848,1.5,11.5,8.915392
65,PHO,Dillon Brooks,SF,NYK,29.196676,32.244088,1.5,19.5,17.181730
18,PHO,Jordan Goodwin,PG,NYK,20.475908,24.165248,1.5,8.5,10.675692
26,PHO,Grayson Allen,SG,NYK,20.778643,23.077472,1.5,11.5,10.105235
16,PHO,Collin Gillespie,PG,NYK,27.690109,28.874202,1.5,12.5,11.118758
76,PHO,Royce O'Neale,SF,NYK,27.693718,31.151266,1.5,9.5,8.563659
102,PHO,Oso Ighodaro,PF,NYK,20.343243,22.318743,1.5,5.5,5.369410


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
77,BRK,Ziaire Williams,SF,LAC,16.865635,20.072887,4.5,6.5,8.382216
60,BRK,Michael Porter Jr.,SF,LAC,32.580784,31.998950,4.5,24.5,26.220947
121,BRK,Day'Ron Sharpe,C,LAC,19.363871,23.425908,4.5,6.5,7.972301
98,BRK,Noah Clowney,PF,LAC,27.195198,29.018104,4.5,11.5,12.527744


../tables/2025/gmday_preds_PTS.csv saved!


In [15]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

98 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
36,OKC,Jalen Williams,SG,MEM,29.743721,29.722245,-5.5,37.5,24.584402
48,OKC,Aaron Wiggins,SG,MEM,24.249126,24.020999,-5.5,24.5,11.936115
44,OKC,Ajay Mitchell,SG,MEM,25.732578,26.531894,-5.5,28.5,17.132118
54,OKC,Alex Caruso,SG,MEM,16.540630,17.293297,-5.5,12.5,7.366850
75,OKC,Luguentz Dort,SF,MEM,25.601997,24.958844,-5.5,15.5,12.574483


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
81,NOP,Zion Williamson,PF,WAS,26.999155,30.571741,-2.5,37.5,25.105433
10,NOP,Jordan Poole,PG,WAS,21.346910,26.947023,-2.5,23.5,13.940255
12,NOP,Jeremiah Fears,PG,WAS,25.235624,26.327991,-2.5,21.5,18.136307
107,NOP,Derik Queen,C,WAS,26.084295,27.126340,-2.5,27.5,25.997402
73,NOP,Trey Murphy III,SF,WAS,34.782333,31.494416,-2.5,30.5,31.714954


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
62,GSW,Jimmy Butler,SF,SAC,22.520132,29.925527,-14.5,31.5,20.256348
19,GSW,De'Anthony Melton,PG,SAC,19.030151,24.937706,-14.5,16.5,11.072546
6,GSW,Stephen Curry,PG,SAC,31.115688,30.641709,-14.5,37.5,33.087173
34,GSW,Brandin Podziemski,SG,SAC,21.004318,23.912698,-14.5,17.5,13.148837
91,GSW,Draymond Green,PF,SAC,24.210556,23.673891,-14.5,21.5,17.755360
92,GSW,Quinten Post,PF,SAC,23.346830,17.054179,-14.5,13.5,16.345692
52,GSW,Gary Payton II,SG,SAC,14.958747,13.337201,-14.5,9.5,11.916393
39,GSW,Moses Moody,SG,SAC,24.293818,24.822355,-14.5,13.5,11.484210
120,GSW,Al Horford,C,SAC,16.309765,16.041448,-14.5,12.5,13.506063


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
3,DEN,Jamal Murray,PG,ATL,23.665972,38.058349,1.5,40.5,29.960495
83,DEN,Aaron Gordon,PF,ATL,20.810839,20.411715,1.5,24.5,16.684462
46,DEN,Christian Braun,SG,ATL,26.726366,24.887005,1.5,20.5,14.244046
37,DEN,Tim Hardaway Jr.,SG,ATL,22.136652,30.475932,1.5,20.5,15.473345
72,DEN,Peyton Watson,SF,ATL,33.887478,35.478035,1.5,26.5,30.056784


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
13,ORL,Anthony Black,PG,PHI,30.920277,31.180814,3.5,26.5,23.912214
82,ORL,Paolo Banchero,PF,PHI,33.946301,34.299194,3.5,37.5,35.680080
24,ORL,Desmond Bane,SG,PHI,32.190823,32.780524,3.5,28.5,28.112589


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
66,LAL,LeBron James,SF,MIL,31.586842,32.184712,-3.5,35.5,38.061928
47,LAL,Marcus Smart,SG,MIL,29.058493,32.480305,-3.5,15.5,17.274300
5,LAL,Luka Doncic,PG,MIL,34.590805,38.044213,-3.5,52.5,51.525928


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
74,WAS,Kyshawn George,SF,NOP,31.030113,30.408756,2.5,22.5,24.827847
29,WAS,Tre Johnson,SG,NOP,25.272772,27.280169,2.5,20.5,18.393259
35,WAS,Bilal Coulibaly,SG,NOP,26.518118,28.627671,2.5,21.5,20.548862
78,WAS,Khris Middleton,SF,NOP,24.528807,20.568049,2.5,17.5,18.063951


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
60,BRK,Michael Porter Jr.,SF,LAC,32.580784,31.998950,4.5,35.5,35.921497
98,BRK,Noah Clowney,PF,LAC,27.195198,29.018104,4.5,17.5,17.817862
121,BRK,Day'Ron Sharpe,C,LAC,19.363871,23.425908,4.5,16.5,16.668036


../tables/2025/gmday_preds_PRA.csv saved!
